In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, updatepivots!, addglobalpivots1sitesweep!, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test

In [28]:
pivotsearch = :full
seed = 123
strictlynested = false

Random.seed!(seed)

R = 20
abstol = 1e-4
δ = 10.0 / 2^R # Peaks are wider than 1/2^R.
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

rindex = [rand(1:2, R) for _ in 1:100]

f(bitlist) = any(bitlist .> 2) ? 0.0 : fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

function fx(x)
    res = exp(-10 * x)
    for r in rpoint
        res += abs(x - r) < δ ? 2 * abstol : 0.0
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=0,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)

MethodError: MethodError: no method matching sweep2site!(::TensorCI2{Float64}, ::typeof(f), ::Int64; iter1::Int64, abstol::Float64, maxbonddim::Int64, pivotsearch::Symbol, strictlynested::Bool, verbosity::Int64, sweepstrategy::Symbol, fillsitetensors::Bool)

Closest candidates are:
  sweep2site!(::TensorCI2{ValueType}, ::Any, ::Int64; iter1, abstol, maxbonddim, sweepstrategy, pivotsearch, verbosity, fillsitetensors) where ValueType got unsupported keyword argument "strictlynested"
   @ TensorCrossInterpolation ~/git/TensorCrossInterpolation.jl/src/tensorci2.jl:808


In [24]:
TCI.addglobalpivots2sitesweep!(tci, f, rindex; fillsitetensors=false)
#TCI.sweep2site!(tci, f, 2; abstol=abstol)

In [25]:
TCI.fillsitetensors!(tci, f)

In [26]:
@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

rindex_test = [rand(1:2, R) for _ in 1:100]

@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex_test]) .> abstol) == 0

Test Passed